In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import statsmodels.api as sm
from statsmodels.stats.sandwich_covariance import cov_hac

In [2]:
#1.select data and data clean
df=pd.read_csv("after GFC.csv")
df.columns=df.columns.str.lower()
cols = ['ret', 'prc', 'shrout']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
print(df['date'].dtype)
df['me'] = df['prc'].abs() * df['shrout']  
df = df.sort_values(['permco', 'date'])

datetime64[ns]


In [3]:
# 2. Filter
df = df[
    (df['prc'].abs() >= 5) &
    (df['shrcd'].isin([10, 11])) &
    (df['exchcd'].isin([1, 2, 3])) &
    (df['ret'].notna())
].copy()
from scipy.stats.mstats import winsorize
winsorize(df['ret'])
df['ret'] = df.groupby('date')['ret'].transform(
    lambda x: winsorize(x, limits=(0.001, 0.001))
)

In [4]:
#The crisis ended in 2009.06, so start the strategy in July 2009
full_date_range = sorted(df['date'].unique())
start_date = pd.Timestamp('2009-07-31') 
J_list = [3, 6, 9, 12]
K_list = [3, 6, 9, 12]
top_n_list = [1500, 3000]
K_max = max(K_list)
all_months = [t for t in full_date_range if t >= start_date]
# Stop opening positions 12 months in advance
all_months = all_months[:-K_max]

In [5]:
# Save df by date grouping into a dictionary, key is the date, and value is the DataFrame of that month
#In this way, don't need to scan the entire DF every time to check the data of a certain month, which greatly improves the speed
gb_date = dict(list(df.groupby('date')))
full_date_range = list(full_date_range)

In [6]:
results_summary = [] #16*2 
results_annual = []  

In [7]:
for top_n in top_n_list:
    for J in J_list:
        for K in K_list:
            print(f"Running: Universe=Top{top_n}, J={J}, K={K}")
            calendar_wml = defaultdict(list)

            for t in all_months:
                t_idx = full_date_range.index(t)

                if t_idx < J + 1:
                    continue
                if t_idx + K >= len(full_date_range):
                    continue

                formation_end   = full_date_range[t_idx - 1]
                formation_start = full_date_range[t_idx - J]

                if formation_end not in gb_date:
                    continue

                eligible_permnos = (
                    gb_date[formation_end]
                    .nlargest(top_n, 'me')['permno']
                    .tolist()
                )

                a = (
                    (df['date'] >= formation_start) &
                    (df['date'] <= formation_end) &
                    (df['permno'].isin(eligible_permnos))
                )
                form_data = df[a]

                stock_counts  = form_data.groupby('permno')['ret'].count()
                valid_permnos = stock_counts[stock_counts == J].index

                if len(valid_permnos) < 100:
                    continue

                cum_ret = (
                    form_data[form_data['permno'].isin(valid_permnos)]
                    .groupby('permno')['ret']
                    .apply(lambda x: np.prod(1 + x) - 1)
                    .reset_index()
                    .rename(columns={'ret': 'cum_ret'})
                )

                cum_ret['decile'] = pd.qcut(cum_ret['cum_ret'], 10, labels=False) + 1
                winners = cum_ret[cum_ret['decile'] == 10]['permno'].tolist()
                losers  = cum_ret[cum_ret['decile'] == 1 ]['permno'].tolist()

                for h in range(1, K + 1):
                    holding_month = full_date_range[t_idx + h]

                    if holding_month not in gb_date:
                        continue

                    hold_data = gb_date[holding_month]
                    w_ret = hold_data[hold_data['permno'].isin(winners)]['ret'].mean()
                    l_ret = hold_data[hold_data['permno'].isin(losers) ]['ret'].mean()

                    if np.isnan(w_ret) or np.isnan(l_ret):
                        continue

                    calendar_wml[holding_month].append(w_ret - l_ret)

            wml_series = pd.Series({
                month: np.mean(vals)
                for month, vals in sorted(calendar_wml.items())
            })

            if len(wml_series) < 12:
                continue

            # ── file1:the summary  ──
            mean_ret = wml_series.mean()
            std_ret  = wml_series.std(ddof=1)#degree of freedom offset
            n        = len(wml_series)

            ols_model = sm.OLS(wml_series.values, np.ones(n)).fit()
            nw_se     = np.sqrt(cov_hac(ols_model, nlags=K))
            t_stat    = mean_ret / nw_se
            sharpe        = (mean_ret / std_ret) * np.sqrt(12)
            cum_ret_total = (1 + wml_series).prod() - 1

            results_summary.append({
                'Universe':    f'Top{top_n}',
                'J':           J,
                'K':           K,
                'N':           n,
                'Mean_Ret(%)': round(mean_ret * 100, 3),
                'T_stat':      round(t_stat.item(), 2),
                'Sharpe':      round(sharpe, 3),
                'CumRet(%)':   round(cum_ret_total * 100, 1),
            })

            # ── file 2 : follow the year ──
            wml_series.index = pd.to_datetime(wml_series.index)
            for year, group in wml_series.groupby(wml_series.index.year):
                yr_mean = group.mean()
                yr_std  = group.std(ddof=1)
                yr_n    = len(group)
                yr_cum  = (1 + group).prod() - 1
                yr_tstat = (yr_mean / (yr_std / np.sqrt(yr_n))) if yr_std != 0 else np.nan
                yr_sharpe = (yr_mean / yr_std) * np.sqrt(12) if yr_std != 0 else np.nan

                results_annual.append({
                    'Universe':    f'Top{top_n}',
                    'J':           J,
                    'K':           K,
                    'Year':        year,
                    'N_months':    yr_n,
                    'Mean_Ret(%)': round(yr_mean * 100, 3),
                    'T_stat':      round(yr_tstat, 2),
                    'Sharpe':      round(yr_sharpe, 3),
                    'CumRet(%)':   round(yr_cum * 100, 1),
                })

summary_df = pd.DataFrame(results_summary)
annual_df  = pd.DataFrame(results_annual)

summary_df.to_csv('momentum_summary_32rows.csv',   index=False)
annual_df.to_csv( 'momentum_annual_32xN rows.csv', index=False)
# ── 4*4 matrix ──
for top_n in top_n_list:
    print(f"\n{'='*50}\nUniverse: Top {top_n}\n{'='*50}")
    subset = summary_df[summary_df['Universe'] == f'Top{top_n}']
    for metric in ['Mean_Ret(%)', 'T_stat', 'Sharpe', 'CumRet(%)']:
        print(f"\n-- {metric} --")
        print(subset.pivot(index='J', columns='K', values=metric).to_string())

Running: Universe=Top1500, J=3, K=3
Running: Universe=Top1500, J=3, K=6
Running: Universe=Top1500, J=3, K=9
Running: Universe=Top1500, J=3, K=12
Running: Universe=Top1500, J=6, K=3
Running: Universe=Top1500, J=6, K=6
Running: Universe=Top1500, J=6, K=9
Running: Universe=Top1500, J=6, K=12
Running: Universe=Top1500, J=9, K=3
Running: Universe=Top1500, J=9, K=6
Running: Universe=Top1500, J=9, K=9
Running: Universe=Top1500, J=9, K=12
Running: Universe=Top1500, J=12, K=3
Running: Universe=Top1500, J=12, K=6
Running: Universe=Top1500, J=12, K=9
Running: Universe=Top1500, J=12, K=12
Running: Universe=Top3000, J=3, K=3
Running: Universe=Top3000, J=3, K=6
Running: Universe=Top3000, J=3, K=9
Running: Universe=Top3000, J=3, K=12
Running: Universe=Top3000, J=6, K=3
Running: Universe=Top3000, J=6, K=6
Running: Universe=Top3000, J=6, K=9
Running: Universe=Top3000, J=6, K=12
Running: Universe=Top3000, J=9, K=3
Running: Universe=Top3000, J=9, K=6
Running: Universe=Top3000, J=9, K=9
Running: Universe=

In [9]:
#focus on 2009 and 2010
df_annual = pd.read_csv('momentum_annual_32xN rows.csv')
yearly_09 = df_annual[df_annual['Year'].isin([2009])]
yearly_10 = df_annual[df_annual['Year'].isin([2010])]
print("--- 2009 Yearly Results ---")
print(yearly_09)
print("--- 2010 Yearly Results ---")
print(yearly_10)

--- 2009 Yearly Results ---
    Universe   J   K  Year  N_months  Mean_Ret(%)  T_stat  Sharpe  CumRet(%)
0    Top1500   3   3  2009         5        1.117    0.79   1.223        5.5
16   Top1500   3   6  2009         5        1.149    0.82   1.264        5.7
32   Top1500   3   9  2009         5        1.149    0.82   1.264        5.7
48   Top1500   3  12  2009         5        1.149    0.82   1.264        5.7
64   Top1500   6   3  2009         5       -0.228   -0.23  -0.352       -1.2
80   Top1500   6   6  2009         5       -0.404   -0.44  -0.677       -2.1
96   Top1500   6   9  2009         5       -0.404   -0.44  -0.677       -2.1
112  Top1500   6  12  2009         5       -0.404   -0.44  -0.677       -2.1
128  Top1500   9   3  2009         5       -2.138   -1.37  -2.125      -10.5
144  Top1500   9   6  2009         5       -2.296   -1.55  -2.408      -11.2
160  Top1500   9   9  2009         5       -2.296   -1.55  -2.408      -11.2
176  Top1500   9  12  2009         5       -2.29